In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'

# DataMap examples

## PVDATA

Get some actual data that we will use to map

In [2]:
import json

In [3]:
rdat = json.load(open('data/live_model_PVDB.json'))

PVDATA = {k:rdat[k]['value'] for k in rdat}

## Klystrons

In [4]:
from lcls_live.datamaps.klystron import KlystronDataMap, klystron_pvinfo, existing_LCLS_klystrons_sector_station

In [5]:
# Get a sector, station that exists
existing_LCLS_klystrons_sector_station[0]

(20, 6)

In [6]:
# This will return a flat dict of info
klystron_pvinfo(20, 6)

{'name': 'K20_6',
 'sector': 20,
 'station': 6,
 'description': 'Klystron in sector 20, station 6 for the GUN',
 'enld_pvname': 'GUN:IN20:1:GN1_AAVG',
 'phase_pvname': 'GUN:IN20:1:GN1_PAVG'}

In [7]:
?KlystronDataMap

Init signature:
KlystronDataMap(
    name: str,
    sector: int,
    station: int,
    description: str = '',
    enld_pvname: str = '',
    phase_pvname: str = '',
    accelerate_pvname: str = '',
    swrd_pvname: str = '',
    stat_pvname: str = '',
    hdsc_pvname: str = '',
    dsta_pvname: str = '',
) -> None
Docstring:     
Attributes
----------
bmad_name : str
pvlist : list[str]

Methods
-------
evaluate(pvdata) :
    Returns
    -------
    dict of:
        enld : float
            energy gain in MeV
        phase : float
            phase in deg
        in_use : bool
            
            
    

as_bmad(pvdata)
File:           ~/Code/GitHub/lcls-live/lcls_live/datamaps/klystron.py
Type:           type
Subclasses:     


In [8]:
# This makes an object
KlystronDataMap(**klystron_pvinfo(20, 6))

KlystronDataMap(name='K20_6', sector=20, station=6, description='Klystron in sector 20, station 6 for the GUN', enld_pvname='GUN:IN20:1:GN1_AAVG', phase_pvname='GUN:IN20:1:GN1_PAVG', accelerate_pvname='', swrd_pvname='', stat_pvname='', hdsc_pvname='', dsta_pvname='')

In [9]:
k = KlystronDataMap(**klystron_pvinfo(20, 6))

# These are the PV names needed to mapping data
k.pvlist


['GUN:IN20:1:GN1_AAVG', 'GUN:IN20:1:GN1_PAVG']

In [10]:
# This will extract those and produce useful information
k.evaluate(PVDATA)

{'enld': 0.000929853773306308, 'phase': 3.8561623144726838, 'in_use': True}

In [11]:
# Actual inputs for a simulation
k.as_bmad(PVDATA)

{'O_K20_6[ENLD_MeV]': 0.000929853773306308,
 'O_K20_6[phase_deg]': 3.8561623144726838,
 'O_K20_6[in_use]': 1}

In [12]:
# Make a large dict to look up a DataMap for any existing klystron by name

KLYSTRON_DATAMAP = {}
for sector, station in existing_LCLS_klystrons_sector_station:
   # print(sector, station)    
    info = klystron_pvinfo(sector, station)
    k = KlystronDataMap(**info)

    KLYSTRON_DATAMAP[info['name']] = k

In [13]:
k = KLYSTRON_DATAMAP['K23_8']
k.pvlist

['KLYS:LI23:81:ENLD',
 'KLYS:LI23:81:PHAS',
 'KLYS:LI23:81:BEAMCODE1_STAT',
 'KLYS:LI23:81:SWRD',
 'KLYS:LI23:81:STAT',
 'KLYS:LI23:81:HDSC',
 'KLYS:LI23:81:DSTA']

In [14]:
# Check that our data is sufficient 

for name, k in KLYSTRON_DATAMAP.items():
    for pv in k.pvlist:
        if pv not in PVDATA:
            print(k.name, pv)